In [ ]:
import os
import logging
from glob import glob
from moviepy.editor import VideoFileClip, AudioFileClip, CompositeAudioClip
from moviepy.audio.fx.all import audio_loop
from moviepy.video.fx.speedx import speedx

# Configure logging
logging.basicConfig(level=logging.WARNING, format='%(levelname)s: %(message)s')

def get_random_file(directory: str, extension: str) -> str:
    files = [f for f in os.listdir(directory) if f.lower().endswith(extension)]
    if not files:
        raise FileNotFoundError(f"No file with extension {extension} found in {directory}")
    return os.path.join(directory, random.choice(files))

def adjust_audio_speed(audio_clip, target_duration):
    current_duration = audio_clip.duration
    if current_duration == target_duration:
        return audio_clip
    speed_factor = current_duration / target_duration
    adjusted = audio_clip.fx(speedx, speed_factor)
    if adjusted.duration < target_duration:
        adjusted = audio_loop(adjusted, duration=target_duration)
    elif adjusted.duration > target_duration:
        adjusted = adjusted.subclip(0, target_duration)
    return adjusted

def assemble_video(i):
    dirs = {
        "middle_videos": "Storage/final_videos/",
        "narrations": "Storage/temp_audios/",
        "music": "Storage/music_library/",
        "final_videos": "Video/final_videos/"
    }
    os.makedirs(dirs["final_videos"], exist_ok=True)
    try:
        middle_video_path = os.path.join(dirs["middle_videos"], f"book_{i}.mp4")
        narration_path = os.path.join(dirs["narrations"], f"voice_{i}.wav")
        song_path = os.path.join(dirs["music"], f"song_{i}.mp3")
        middle_clip = VideoFileClip(middle_video_path)
        video_duration = middle_clip.duration
        narration_audio = AudioFileClip(narration_path)
        song_audio = AudioFileClip(song_path)
        adjusted_narration = adjust_audio_speed(narration_audio, video_duration)
        music_audio = audio_loop(song_audio, duration=video_duration).volumex(0.2)
        composite_audio = CompositeAudioClip([music_audio, adjusted_narration])
        final_clip = middle_clip.set_audio(composite_audio)
        final_video_path = os.path.join(dirs["final_videos"], f"video_{i}.mp4")
        final_clip.write_videofile(
            final_video_path,
            codec="libx264",
            audio_codec="aac",
            temp_audiofile="temp-audio.m4a",
            remove_temp=True,
            verbose=False
        )
        logging.info(f"Video saved to {final_video_path}")
        middle_clip.close()
        narration_audio.close()
        song_audio.close()
        final_clip.close()
    except Exception as e:
        logging.error(f"An error occurred during video assembly for book {i}: {e}")

def main():
    # Ensure directories exist
    os.makedirs("Storage/final_videos", exist_ok=True)
    os.makedirs("Storage/temp_audios", exist_ok=True)
    os.makedirs("Storage/music_library", exist_ok=True)
    os.makedirs("Video/final_videos", exist_ok=True)

    # Use glob to dynamically find all video files matching "book_*.mp4"
    video_files = glob("Storage/final_videos/book_*.mp4")

    for video_file in video_files:
        # Extract the number from the filename, e.g., "book_4.mp4" -> "4"
        base = os.path.basename(video_file)
        try:
            book_number = base.split("_")[1].split(".")[0]
            logging.info(f"Processing video for book {book_number}")
            assemble_video(book_number)
        except Exception as e:
            logging.error(f"Failed to process video for {base}: {e}")

if __name__ == '__main__':
    main()
